In [ ]:
# Setting up the environment
from pyspark import SparkContext
from functools import reduce
from collections import Counter
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as PySparkFunctions
from pyspark.sql.functions import *
from pyspark.sql.window import Window
from itertools import chain
import statistics
# sc.stop() #stop previous session
sc = SparkContext("local", "URRDD") #create an instance of SparkContext


In [ ]:
# reading from stopwrods file
f = open("/home/jovyan/Data/stopwords_en.txt","r")
global stopwords
# split each line
stopwords = f.read().splitlines()
#  function to find top ten occuring words related to each user and excluding stop words
def most_frequent(List,stopwords):
    words = []
    for word in List:
        if word not in stopwords:
            if word != '' :
                words.append(word)
    occurence_count = Counter(words)    
    top_ten = occurence_count.most_common(10)
    return [x[0] for x in top_ten] #we don't want the count, so returning only the words
# definig min and max functions
def find_max(x,y):
    if x <= y : return y
    else: return x
def find_min(x,y):
    if x <= y : return x
    else: return y

In [ ]:
# Exercise 1.2.a
lines = sc.textFile("/home/jovyan/Data/users_libraries.txt") #read lines from user_libraries.txt file as RDD
user_liked = lines.map(lambda x: (x.split(";")[0], x.split(";")[1:] )) #create pair by applying a lambda function and map lines to it
user_liked.collect() #collecting created pair and checking if RDD's correctly generated

In [ ]:
# Exercise 1.2.b
import re
lines = sc.textFile("/home/jovyan/Data/papers.csv") #read lines from papers.csv file as RDD
# splitting the lines read from csv files by comma. creating a new RDD from that
paper_words_inter = lines.map(lambda x: (x.split(",")[0], [reduce(lambda i, j: i + j, x.split(',')[13:])][0]))
# Creating a RDD which contains tuples of (paper_id,[words_in_it]) spliting by space and " .
paper_words = paper_words_inter.map(lambda x: (x[0],re.split(' |"',x[1])))
paper_words.collect()

In [ ]:
# Exercise 1.3

# ! ATTENTION !
# ------------------------------------------------------------------------------------------------------
# "for each user the top-10 most frequent words appearing in the papers she
# likes. Exclude the stop words listed in stopwords en.txt" 
# I assumed the above statement in project document referes to words included in both title and abstarct.
# Since it does not clarify which one and my question in the forum regarding that remained unanswerd
# ------------------------------------------------------------------------------------------------------
user_liked_splited = user_liked.map(lambda x: (x[0],x[1][0].split(","))) 
# creating a new RDD which each user only liked one paper and keys are not unique.in order
# to be able to join
user_liked_flated = user_liked_splited.flatMapValues(lambda x: x).collect()
for i in range(len(user_liked_flated)):
    user_liked_flated[i] = (user_liked_flated[i][1],user_liked_flated[i][0]) 
    # swaping each item key and value for join transformation     
#convert to RDD after swaping and join two RDDs
user_liked_words = sc.parallelize(user_liked_flated).join(paper_words)
#Creating a RDD contains each user and words in each of his/her liked papers. 
# in this RDD (user_words) we have reaccuring user_ids
user_words = user_liked_words.map(lambda x: x[1])
# using reduce by key to combine all words related to a user to it's unique ID, 
# now user_ids are unique in user_combined_words
user_combined_words = user_words.reduceByKey(lambda x,y: x+y)

# mapping RDD to get a new RDD called user_top_ten which contains (user_id, top_ten_words) 
user_top_ten = user_combined_words.map(lambda x: (x[0], most_frequent(x[1],stopwords)))
user_top_ten.collect()

In [ ]:
# Exercise 1.4.a)
number_of_users = user_liked.groupByKey().count()
number_of_papers = paper_words.groupByKey().count()
# Number of "distinct" user_ids(or users by meaning)
print("Number of distinct users is:", number_of_users) 
print("Number of distinct papers is:",number_of_papers) 
# Number of "distinct" paper_ids(or papers by meaning)

ratings_rdd = user_liked.map(lambda x: x[1][0].split(",")) 
# spliting each liked paper(since it is a string)
number_of_ratings_rdd = ratings_rdd.map(lambda x: len(x)) 
# we get each t=(user,[liked_papers]) t[1], so now this new rdd contains number of papers each user liked
ratings_count = number_of_ratings_rdd.reduce(lambda x,y : x+y) 
# using reduce action to sum every index of this rdd and printing as result
print("Number of all ratings is:",ratings_count)

# Exercise 1.4.b)

#using the rdd we created earlier which contains number of liked papers 
#for each user at every index and reduce action to find minimum number, here I used a function find_min()
min_ratings = number_of_ratings_rdd.reduce(lambda x,y: find_min(x,y)) 
print("Minimum number of rating a user has given is:",min_ratings)

# Exercise 1.4.c)

#using the rdd we created earlier which contains number of liked papers 
#for each user at every index and reduce action to find maximum number, here I used a function find_max()
max_ratings = number_of_ratings_rdd.reduce(lambda x,y: find_max(x,y))
print("Maximum number of rating a user has given is:",max_ratings)

# Exercise 1.4.d)
print("Average number of user ratings is:",ratings_count/number_of_users)

# Exercise 1.4.e)
number_of_ratings_rdd_list = number_of_ratings_rdd.collect() 
# now that we have a RDD which each item of it is the number of papers each user liked,
# we can easily use python statistics library to find the standard deviation of them.
print(statistics.stdev(number_of_ratings_rdd_list))

In [ ]:
# each paper(not unique) and users who rated it
paper_users = user_liked_words.map(lambda x: (x[0],x[1][0]))
# paper_combined_users contains each paper and a tuple of users who rated it. this time papers are unique
# I did this using reduceByKey and adding a "|" in between so we can count later
paper_combined_users = paper_users.reduceByKey(lambda x,y:x+"|"+y)
# counting | and adding one since the number of splitter is one less than theb number of users or 
# by meaning ratings for each paper
# paper_number_of_rates contains  number of ratings for each unique paper, (I removed the paper_id since we
# don't need it)
paper_rates_rdd  = paper_combined_users.map(lambda x: x[1].count("|")+1)
# Exercise 1.4.f)
#using the rdd we created earlier which contains number of ratings 
#for each paper at every index and reduce action to find minimum number, here I used a function find_min()
min_no_ratings = paper_rates_rdd.reduce(lambda x,y: find_min(x,y)) 
print("Minimum number of rating an item has received is:",min_no_ratings)
# Exercise 1.4.g)
#using the rdd we created earlier which contains number of ratings 
#for each paper at every index and reduce action to find minimum number, here I used a function find_min()
max_no_ratings = paper_rates_rdd.reduce(lambda x,y: find_max(x,y)) 
print("Maximum number of rating an item has received is:",max_no_ratings)
# Exercise 1.4.h)
print("Average number of ratings of items is:",ratings_count/number_of_papers)
# Exercise 1.4.i)
paper_rates_rdd_list = paper_rates_rdd.collect() 
# now that we have a RDD which each item of it is the number of papers each user liked,
# we can easily use python statistics library to find the standard deviation of them.
print(statistics.stdev(paper_rates_rdd_list))

In [ ]:
# Exercise 1.5
spark = SparkSession.builder.appName("DataFrame").getOrCreate()
# Defining an appropriate schema for dataset
userSchema = StructType([
    StructField("user_id", StringType(), True),        
    StructField("liked_papers", StringType(), True)
])
# read from file
user_paper_df = spark.read.load("/home/jovyan/Data/users_libraries.txt", format="csv", header="false", sep=';', schema=userSchema)
user_paper_df.show()
#----------------------------------------------------------------------------
# Defining an appropriate schema for dataset
paperSchema = StructType([
    StructField("paper_id", StringType(), False),        
    StructField("type", StringType(), True),
    StructField("journal", StringType(), True),
    StructField("book_title", StringType(), True),
    StructField("series", StringType(), True),
    StructField("publisher", StringType(), True),
    StructField("pages", StringType(), True),
    StructField("volume", StringType(), True),
    StructField("number", StringType(), True),
    StructField("year", StringType(), True),
    StructField("month", StringType(), True),
    StructField("postedat", StringType(), True),
    StructField("address", StringType(), True),
    StructField("title", StringType(), True),
    StructField("abstract", StringType(), True),
])
# read from file
papers_df = spark.read.load("/home/jovyan/Data/papers.csv", format="csv", header="false", sep=',', schema=paperSchema)
papers_df.show()

In [ ]:
# Exercise 1.6 first part. task 1.3 for data frames
# first let's select desired columns and get rid of others
paper_content_df = papers_df.select(col("paper_id"),col("title"),col("abstract"))
# contructing a df containing user_ids and liked papers, seperated by ","
df = user_paper_df.select("user_id",PySparkFunctions.split("liked_papers", ",").alias("papers"))
# using explode to flatten the dataframe
# in other words; it assings each paper id to one user_id
paperid_userid = df.select(explode("papers").alias("paper"),"user_id")
# contructing a df containing paper_ids and its content which contains title+abstarct
paperid_content=paper_content_df.select("paper_id",concat("title","abstract").alias("content"))
# now we can join (paperid,userid) and (paperid_content)
paperid_userid_content= paperid_userid.join(paperid_content,paperid_userid.paper ==  paperid_content.paper_id,"inner")
# selecting desired columns user_id and content and spliting the strings in content by seperators
userid_words = paperid_userid_content.select("user_id",PySparkFunctions.split("content", ' |;|"|\\{|\\}').alias("words"))

In [ ]:
# using groupBy to combine all words which are in papers a user liked together using aggregate function and collect_list
userid_allwords = userid_words.groupBy('user_id').agg(PySparkFunctions.collect_list("words").alias("words"))
# defining a function which first flattens the list of words related to user
# and then find the top ten frequent ones
def flaten_find_most_frequent(z):
    flatten_list = list(chain.from_iterable(z))
    return most_frequent(flatten_list,stopwords)
# constructing a udf function
convertUDF = udf(lambda z: flaten_find_most_frequent(z),StringType())
# applying the udf function on words column
userid_topten = userid_allwords.select(col("user_id"),convertUDF(col("words")).alias("top_ten_words"))
userid_topten.show(truncate=False)

In [ ]:
# Exercise 1.6 first part. task 1.4 for dataframes
# a)
print("Number of distinct users is:", user_paper_df.count()) 
print("Number of distinct papers is:",papers_df.count()) 
print("Number of ratings is:",paperid_userid.count())
# b)
# defining a udf to return number of liked papers for each user
convertUDF = udf(lambda x: len(x),IntegerType())
# applying udf to df which was (user_ids and liked papers, seperated by ",")
user_ratedcount = df.select(col("user_id"),convertUDF(col("papers")).alias("rated_count"))
# using aggregate function to get min
min_ratings = user_ratedcount.agg({"rated_count": "min"}).collect()[0][0]
print("Minimum number of rating a user has given is:",min_ratings)
# c) the same for max
max_ratings = user_ratedcount.agg({"rated_count": "max"}).collect()[0][0]
print("Maximum number of rating a user has given is:",max_ratings)
# d) and the same for average
avg_ratings = user_ratedcount.agg({"rated_count": "avg"}).collect()[0][0]
print("Average number of rating a user has given is:",avg_ratings)
# e) and the same for standard deviation
stddev_ratings = user_ratedcount.agg({"rated_count": "stddev"}).collect()[0][0]
print("Standard deviation of number of rating a user has given is:",stddev_ratings)

In [ ]:
# f)
# collecting users who liked for each paper
paperid_users = paperid_userid.groupBy('paper').agg(PySparkFunctions.collect_list("user_id").alias("users_who_liked"))
# create a udf to get len of a column
convertUDF = udf(lambda x: len(x),IntegerType())
# creating df containing paper id and number of users who liked this paper
paper_usercount = paperid_users.select(col("paper"),convertUDF(col("users_who_liked")).alias("users_count"))
# no we can find min,max.avg and stddev using this new df 
# using aggregate function to get min
min_ratings = paper_usercount.agg({"users_count": "min"}).collect()[0][0]
print("Minimum number of ratings an item has received is:",min_ratings)
# c) the same for max
max_ratings = paper_usercount.agg({"users_count": "max"}).collect()[0][0]
print("Maximum number of ratings an item has received is:",max_ratings)
# d) and the same for average
avg_ratings = paper_usercount.agg({"users_count": "avg"}).collect()[0][0]
print("Average number of ratings an item has received is:",avg_ratings)
# e) and the same for standard deviation
stddev_ratings = paper_usercount.agg({"users_count": "stddev"}).collect()[0][0]
print("Standard deviation number of ratings an item has received is:",stddev_ratings)